In [ ]:
import json
import sys
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import pytz
import time
from joblib import load
import logging
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
import talib
import os
import threading
import subprocess
import psutil
import warnings
import numpy as np

warnings.filterwarnings("ignore", message="Dependency not available for matrix_profile")

# Initialize the lock globally
mt5_lock = threading.Lock()

# Configure logging
logging.basicConfig(
    filename='trading_EURUSD_GBPUSD_USDCAD_AUDUSD_USDCHF_Buy_Sell_D1.log', 
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Kill any existing MetaTrader 5 instances
def kill_mt5_instances():
    for process in psutil.process_iter(['name']):
        try:
            if "terminal64.exe" in process.info['name'].lower():
                process.kill()
                logging.info(f"Killed existing MT5 instance: PID {process.pid}")
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass  # Ignore these exceptions

# Initialize connection to MetaTrader 5 with proper locking and multiple instances handling
def init_mt5_connection(login, password, server, terminal_path):
    with mt5_lock:
        mt5.shutdown()
        time.sleep(5)

        # Start MetaTrader 5 terminal for the specific account
        subprocess.Popen([terminal_path, '/portable', '/login', str(login), '/password', password, '/server', server])
        time.sleep(10)

        for _ in range(30):  # Poll for 30 seconds, check if MT5 is running
            if mt5.initialize(login=login, password=password, server=server):
                logging.info(f"Connected to MetaTrader 5 with login: {login}")
                print(f"Connected to MetaTrader 5 with login: {login}")
                return True
            time.sleep(1)

        logging.error(f"initialize() failed, error code = {mt5.last_error()}")
        sys.exit("MT5 initialization failed.")

# Fetch historical data
def fetch_historical_data(symbol, timeframe, start_date, end_date):
    with mt5_lock:
        data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    
    # Check if data is empty or None
    if data is None or len(data) == 0:
        logging.error(f"No historical data found for {symbol} in the given date range.")
        print(f"No historical data found for {symbol}.")
        return None
    
    ohlc_data = pd.DataFrame(data)
    if 'time' not in ohlc_data.columns:
        logging.error(f"'time' column missing in the fetched data for {symbol}.")
        print(f"Error: 'time' column missing in the fetched data for {symbol}.")
        return None
    
    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s', utc=True)
    return ohlc_data[['time', 'open', 'high', 'low', 'close']]

# Calculate mean candle size dynamically
def calculate_mean_candle_size(df):
    df['candle_size'] = df['high'] - df['low']
    mean_candle_size = df['candle_size'].mean()
    return mean_candle_size

# Add rolling features
def add_rolling_features(df, window):
    df['rolling_mean_open'] = df['open'].rolling(window=window).mean()
    df['rolling_std_open'] = df['open'].rolling(window=window).std()
    df['rolling_mean_close'] = df['close'].rolling(window=window).mean()
    df['rolling_std_close'] = df['close'].rolling(window=window).std()
    df['rolling_mean_high'] = df['high'].rolling(window=window).mean()
    df['rolling_std_high'] = df['high'].rolling(window=window).std()
    df['rolling_mean_low'] = df['low'].rolling(window=window).mean()
    df['rolling_std_low'] = df['low'].rolling(window=window).std()
    return df

# Add lag features
def add_lag_features(df, lags):
    for lag in lags:
        df[f'open_lag_{lag}'] = df['open'].shift(lag)
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'high_lag_{lag}'] = df['high'].shift(lag)
        df[f'low_lag_{lag}'] = df['low'].shift(lag)
    return df

# Calculate indicators
def calculate_indicators(df):
    # EMAs
    df['EMA_9'] = talib.EMA(df['close'], timeperiod=9)
    df['EMA_21'] = talib.EMA(df['close'], timeperiod=21)
    df['EMA_50'] = talib.EMA(df['close'], timeperiod=50)
    
    # RSI
    df['RSI_9'] = talib.RSI(df['close'], timeperiod=9)
    df['RSI_14'] = talib.RSI(df['close'], timeperiod=14)
    df['RSI_21'] = talib.RSI(df['close'], timeperiod=21)
    
    # WILLR
    df['WILLR_15'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=15)
    df['WILLR_23'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=23)
    df['WILLR_42'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=42)
    df['WILLR_145'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=145)
    
    # SAR
    df['SAR'] = talib.SAR(df['high'], df['low'], acceleration=0.02, maximum=0.2)
    
    # Bollinger Bands
    df['BB_upper'], df['BB_middle'], df['BB_lower'] = talib.BBANDS(
        df['close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0
    )
    df['BB_width'] = df['BB_upper'] - df['BB_lower']
    
    # MACD
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(
        df['close'], fastperiod=12, slowperiod=26, signalperiod=9
    )
    
    # CCI
    df['CCI_14'] = talib.CCI(df['high'], df['low'], df['close'], timeperiod=14)
    
    return df

# Optional helper: example of how you might extract rolling features for a single column
def extract_rolling_features(df, column_name, symbol):
    df_melted = df[['time', column_name]].copy()
    df_melted["Symbols"] = symbol
    # Roll the data in a time-series manner
    df_rolled = roll_time_series(
        df_melted, 
        column_id="Symbols", 
        column_sort="time", 
        max_timeshift=20, 
        min_timeshift=5
    )
    # Extract features
    X = extract_features(
        df_rolled.drop("Symbols", axis=1),
        column_id="id",
        column_sort="time",
        column_value=column_name,
        impute_function=impute,
        show_warnings=False
    )
    # Map index back to original time
    X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
    return X

# Process data
def process_data_for_features(df, symbol):
    df = add_rolling_features(df, window=5)
    df = add_lag_features(df, lags=[1, 2, 3, 4, 5])
    df = df.dropna().reset_index(drop=True)

    # Extract rolling features
    X1  = extract_rolling_features(df, 'WILLR_15', symbol)
    X2  = extract_rolling_features(df, 'WILLR_42', symbol)
    X3  = extract_rolling_features(df, 'RSI_14',  symbol)
    X4  = extract_rolling_features(df, 'MACD_hist', symbol)
    X5  = extract_rolling_features(df, 'EMA_9', symbol)
    X6  = extract_rolling_features(df, 'EMA_21', symbol)
    X7  = extract_rolling_features(df, 'EMA_50', symbol)
    X9  = extract_rolling_features(df, 'RSI_9', symbol)
    X10 = extract_rolling_features(df, 'RSI_21', symbol)
    X11 = extract_rolling_features(df, 'WILLR_23', symbol)
    X12 = extract_rolling_features(df, 'WILLR_145', symbol)
    X13 = extract_rolling_features(df, 'SAR', symbol)
    X14 = extract_rolling_features(df, 'BB_width', symbol)
    X15 = extract_rolling_features(df, 'MACD_signal', symbol)
    X16 = extract_rolling_features(df, 'CCI_14', symbol)

    X = pd.concat(
        [X1, X2, X3, X4, X5, X6, X7, X9, X10, X11, X12, X13, X14, X15, X16],
        axis=1, 
        join='inner'
    ).dropna()

    df['time'] = pd.to_datetime(df['time'], utc=True)
    df = df.set_index('time')

    # Align X with df’s index
    X = X[X.index.isin(df.index)]
    X = pd.concat([df, X], axis=1, join='inner')

    return X

def fetch_and_process_data(symbol, timeframe, signals, selected_features, start_date, end_date):
    logging.info(f"Fetching and processing data for {symbol}")

    df = fetch_historical_data(symbol, timeframe, start_date, end_date)
    if df is None or df.empty:
        logging.error(f"No historical data found for {symbol}.")
        return None, None

    mean_candle_size = calculate_mean_candle_size(df)
    df = calculate_indicators(df)
    combined_df = process_data_for_features(df, symbol)

    missing_features = [feat for feat in selected_features if feat not in combined_df.columns]
    for feat in missing_features:
        combined_df[feat] = 0

    combined_df = combined_df.dropna()
    return combined_df, mean_candle_size

# Calculate prices based on risk-reward ratio
def calculate_prices(entry_price, risk_reward_ratio, mean_candle_size, trade_type):
    try:
        risk_part, reward_part = map(float, risk_reward_ratio.split(':'))
    except ValueError:
        logging.error(f"Invalid risk_reward_ratio format: {risk_reward_ratio}. Expected format 'X:Y'.")
        raise ValueError(f"Invalid risk_reward_ratio format: {risk_reward_ratio}. Expected format 'X:Y'.")
    
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part

    if trade_type == "Buy":
        sl_price = entry_price - risk_amount
        tp_price = entry_price + reward_amount
    elif trade_type == "Sell":
        sl_price = entry_price + risk_amount
        tp_price = entry_price - reward_amount
    else:
        raise ValueError(f"Invalid trade_type: {trade_type}. It must be 'Buy' or 'Sell'.")

    return sl_price, tp_price

# Place order with retry mechanism
def place_order(symbol, volume, sl_price, tp_price, trade_type, config, retry_attempts=3):
    for attempt in range(retry_attempts):
        with mt5_lock:
            tick_info = mt5.symbol_info_tick(symbol)
            if tick_info is None:
                logging.error(f"Failed to retrieve tick information for {symbol}.")
                return
            price = tick_info.ask if trade_type == "Buy" else tick_info.bid
            order_type = mt5.ORDER_TYPE_BUY if trade_type == "Buy" else mt5.ORDER_TYPE_SELL
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": volume,
                "type": order_type,
                "price": price,
                "sl": sl_price,
                "tp": tp_price,
                "deviation": 10,
                "magic": config["magic"],
                "comment": f"Python script {trade_type} order",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }

            result = mt5.order_send(request)
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                logging.info(f"{trade_type} order placed successfully for {symbol}.")
                print(f"{trade_type} order placed successfully for {symbol}.")
                return True
            else:
                logging.error(f"Order placement failed for {symbol}: {result.retcode}. Retrying...")
                print(f"Order placement failed for {symbol}: {result.retcode}. Retrying...")

        time.sleep(2)

    logging.error(f"Order failed after {retry_attempts} attempts for {symbol}")
    return False

# Predict and gather predictions
def predict_and_gather_predictions(scaler, model, symbol, timeframe, start_date, end_date, features, trade_type, threshold=None, pca=None):
    """
    If pca is not None, apply pca.transform() after scaler.transform().
    """
    logging.info(f"Generating predictions for {symbol} - {trade_type}")

    signals = ['WILLR_15', 'WILLR_42']  # example signals or placeholders
    df_processed, mean_candle_size = fetch_and_process_data(symbol, timeframe, signals, features, start_date, end_date)
    if df_processed is None:
        logging.error(f"No valid data for {symbol}. Skipping predictions.")
        return None, mean_candle_size

    if df_processed.empty:
        logging.error(f"Dataframe empty after shifting for {symbol} - {trade_type}.")
        return None, mean_candle_size

    # Extract features
    X = df_processed.drop(columns=['b_flag'])
    
    # Scale features
    scaled_data = scaler.transform(X)

    # Apply PCA if provided
    if pca is not None:
        scaled_data = pca.transform(scaled_data)

    # Predict probabilities
    probas = model.predict_proba(scaled_data)[:, 1]
    custom_threshold = threshold if threshold is not None else 0.5
    y_pred = (probas >= custom_threshold).astype(int)
    
    # Save predictions to CSV
    df_pred = pd.DataFrame({
        'time': df_processed.index,
        'prediction': y_pred
    })
    df_pred.to_csv(f'pred_{symbol}_{trade_type}.csv', index=False)
    logging.info(f"Predictions saved to pred_{symbol}_{trade_type}.csv")
    
    # Return the latest prediction
    pred = y_pred[-1] if len(y_pred) > 0 else 0
    return pred, mean_candle_size

# Decide and execute trades based on predictions
def decide_and_trade(buy_pred, sell_pred, symbol, volume, risk_reward_ratio, mean_candle_size, config):
    if buy_pred == 1 and sell_pred == 1:
        logging.info(f"Both Buy and Sell predictions are 1 for {symbol}. No trade will be taken.")
        print(f"Both Buy and Sell predictions are 1 for {symbol}. No trade will be taken.")
        return

    if buy_pred == 1 and sell_pred == 0:
        entry_price = mt5.symbol_info_tick(symbol).ask
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size, "Buy")
        place_order(symbol, volume, sl_price, tp_price, "Buy", config)

    if sell_pred == 1 and buy_pred == 0:
        entry_price = mt5.symbol_info_tick(symbol).bid
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size, "Sell")
        place_order(symbol, volume, sl_price, tp_price, "Sell", config)

# Process each currency pair
def process_pair(config, utc_from, utc_to, trade_decisions):
    symbol = config['symbol']
    logging.info(f"Processing pair: {symbol} for Buy and Sell")

    buyscaler, buypca, buymodel = None, None, None
    sellscaler, sellpca, sellmodel = None, None, None

    # Load Buy model components
    try:
        buyscaler = load(config["buy_scaler_path"])
        buymodel  = load(config["buy_model_path"])
        with open(config["buy_features_path"], 'r') as f:
            buy_features = json.load(f)
        if config.get("buy_pca_path"):
            buypca = load(config["buy_pca_path"])
    except Exception as e:
        logging.warning(f"Buy model/scaler/PCA missing for {symbol}: {e}")
        print(f"Buy model/scaler/PCA missing for {symbol}: {e}")
        buy_features = []

    # Load Sell model components
    try:
        if "sell_scaler_path" in config and "sell_model_path" in config:
            sellscaler = load(config["sell_scaler_path"])
            sellmodel  = load(config["sell_model_path"])
            with open(config["sell_features_path"], 'r') as f:
                sell_features = json.load(f)
            if config.get("sell_pca_path"):
                sellpca = load(config["sell_pca_path"])
        else:
            logging.warning(f"Sell model or scaler missing for {symbol}")
            print(f"Sell model or scaler missing for {symbol}")
            sell_features = []
    except Exception as e:
        logging.warning(f"Error loading sell model for {symbol}: {e}")
        print(f"Error loading sell model for {symbol}: {e}")
        sell_features = []

    # Initialize MT5 connection
    try:
        with mt5_lock:
            init_mt5_connection(config['login'], config['password'], config['server'], config['terminal_path'])
    except Exception as e:
        logging.error(f"Failed to initialize MT5 connection for {symbol}: {e}")
        return

    try:
        while True:
            current_time = datetime.now(pytz.utc)

            # Example trading window: 14:58 UTC daily
            if current_time.hour == 21 and current_time.minute == 54:
                buy_pred, mean_candle_size_buy = None, None
                sell_pred, mean_candle_size_sell = None, None

                # Handle Buy Predictions
                if buyscaler is not None and buymodel is not None and len(buy_features) > 0:
                    buy_pred, mean_candle_size_buy = predict_and_gather_predictions(
                        scaler=buyscaler,
                        model=buymodel,
                        symbol=symbol,
                        timeframe=config["timeframe"],
                        start_date=utc_from,
                        end_date=utc_to,
                        features=buy_features,
                        trade_type="Buy",
                        threshold=config.get("buy_threshold", None),
                        pca=buypca
                    )

                # Handle Sell Predictions
                if sellscaler is not None and sellmodel is not None and len(sell_features) > 0:
                    sell_pred, mean_candle_size_sell = predict_and_gather_predictions(
                        scaler=sellscaler,
                        model=sellmodel,
                        symbol=symbol,
                        timeframe=config["timeframe"],
                        start_date=utc_from,
                        end_date=utc_to,
                        features=sell_features,
                        trade_type="Sell",
                        threshold=config.get("sell_threshold", None),
                        pca=sellpca
                    )

                # Decide and execute trades
                if buy_pred is not None and sell_pred is not None:
                    # Determine which mean_candle_size to use
                    if config.get("use_dynamic_mean_candle_size", True):
                        # Prefer dynamic mean_candle_size if both are available
                        if mean_candle_size_buy is not None and mean_candle_size_sell is not None:
                            mean_candle_size = (mean_candle_size_buy + mean_candle_size_sell) / 2
                        elif mean_candle_size_buy is not None:
                            mean_candle_size = mean_candle_size_buy
                        elif mean_candle_size_sell is not None:
                            mean_candle_size = mean_candle_size_sell
                        else:
                            logging.error(f"Unable to determine mean_candle_size for {symbol}.")
                            mean_candle_size = config.get("mean_candle_size", 0.005)  # Fallback
                    else:
                        mean_candle_size = config.get("mean_candle_size", 0.005)
                    
                    decide_and_trade(
                        buy_pred=buy_pred,
                        sell_pred=sell_pred,
                        symbol=symbol,
                        volume=config["volume"],
                        risk_reward_ratio=config["buy_risk_reward_ratio"],  # Assuming same RR for buy and sell
                        mean_candle_size=mean_candle_size,
                        config=config
                    )

                    logging.info(f"Finished trading for {symbol} at {current_time} UTC")

            time.sleep(60)  # Wait for 1 minute before checking again

    except KeyboardInterrupt:
        logging.info(f"Script terminated by user for {symbol}.")
        print(f"Script terminated by user for {symbol}.")
    finally:
        with mt5_lock:
            mt5.shutdown()
        logging.info(f"MetaTrader 5 connection closed for {symbol}.")
        print(f"MetaTrader 5 connection closed for {symbol}.")

# Main function to initiate trading threads
def main():
    kill_mt5_instances()

    # Example configs with buy/sell thresholds specified
    eurusd_config = {
        "symbol": "EURUSD",
        "login": 52118058,
        "password": "tqLuM0wom!pL@P",
        "server": "ICMarketsEU-Demo",
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU5\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "1:2",
        "sell_risk_reward_ratio": "1:1",

        # Thresholds for buy/sell
        "buy_threshold": 0.6,
        "sell_threshold": 0.5,

        "use_dynamic_mean_candle_size": True,  # Set to False to use hardcoded mean_candle_size
        "mean_candle_size": 0.005,             # Used only if use_dynamic_mean_candle_size is False

        "buy_scaler_path": "EURUSD_D1_2023buy/scaler.joblib",
        "buy_model_path": "EURUSD_D1_2023buy/model.joblib",
        "buy_pca_path":   'EURUSD_D1_2023buy/pca.joblib',
        "sell_scaler_path": "EURUSD_D1_3112sell/scaler.joblib",
        "sell_model_path": "EURUSD_D1_3112sell/model.joblib",
        # "sell_pca_path":  'EURUSD_D1_3112sell/pca.joblib',
        "buy_features_path": "EURUSD_D1_2023buy/feature_names.json",
        "sell_features_path": "EURUSD_D1_3112sell/feature_names.json",
        "magic": 1001,
    }

    gbpusd_config = {
        "symbol": "GBPUSD",
        "login": 52118161,
        "password": "t0cecVH!vqselU",
        "server": "ICMarketsEU-Demo",
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU6\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "2:2",
        "sell_risk_reward_ratio": "1:1",

        # Thresholds for buy/sell
        "buy_threshold": 0.6,
        "sell_threshold": 0.45,

        "use_dynamic_mean_candle_size": False,  # Set to True to calculate dynamically
        "mean_candle_size": 0.01,               # Used only if use_dynamic_mean_candle_size is False

        "buy_scaler_path": "GBPUSD_D1_2023buy/scaler.joblib",
        "buy_model_path": "GBPUSD_D1_2023buy/model.joblib",
        "buy_pca_path": "GBPUSD_D1_2023buy/pca.joblib",
        "sell_scaler_path": "GBPUSD_D1_3112sellfinal/scaler.joblib",
        "sell_model_path": "GBPUSD_D1_3112sellfinal/model.joblib",
        # "sell_pca_path":  'GBPUSD_D1_3112sellfinal/pca.joblib',
        "buy_features_path": "GBPUSD_D1_2023buy/feature_names.json",
        "sell_features_path": "GBPUSD_D1_3112sellfinal/feature_names.json",
        "magic": 1002,
    }

    usdcad_config = {
        "symbol": "USDCAD",
        "login": 52118215,
        "password": "JU3fQ$XZ5FLeqQ",
        "server": "ICMarketsEU-Demo",
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU7\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "1:1",
        "sell_risk_reward_ratio": "2:3",

        # Thresholds for buy/sell
        "buy_threshold": 0.6,
        "sell_threshold": 0.6,

        "use_dynamic_mean_candle_size": True,   # Set to False to use hardcoded mean_candle_size
        "mean_candle_size": 0.0085,             # Used only if use_dynamic_mean_candle_size is False

        "buy_scaler_path": "USDCAD_D1_2023buy/scaler.joblib",
        "buy_model_path": "USDCAD_D1_2023buy/model.joblib",
        "buy_pca_path":   'USDCAD_D1_2023buy/pca.joblib',
        "sell_scaler_path": "USDCAD_D1_3112_Sell/scaler.joblib",
        "sell_model_path": "USDCAD_D1_3112_Sell/model.joblib",
        # "sell_pca_path":  'USDCAD_D1_3112_Sell/pca.joblib',
        "buy_features_path": "USDCAD_D1_2023buy/feature_names.json",
        "sell_features_path": "USDCAD_D1_3112_Sell/feature_names.json",
        "magic": 1003,
    }

    audusd_config = {
        "symbol": "AUDUSD",
        "login": 52118219,
        "password": "Taz$Ci50wB1SEx",
        "server": "ICMarketsEU-Demo",
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU8\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "1:1",
        "sell_risk_reward_ratio": "1:1",

        # Thresholds for buy/sell
        "buy_threshold": 0.6,
        "sell_threshold": 0.55,

        "use_dynamic_mean_candle_size": False,  # Set to True to use hardcoded mean_candle_size
        "mean_candle_size": 0.008,               # Used only if use_dynamic_mean_candle_size is False

        "buy_scaler_path": "AUDUSD_D1_2023buy/scaler.joblib",
        "buy_model_path": "AUDUSD_D1_2023buy/model.joblib",
        "buy_pca_path": "AUDUSD_D1_2023buy/pca.joblib",
        
        "sell_scaler_path": "AUDUSD_D1_3112sell/scaler.joblib",
        "sell_model_path": "AUDUSD_D1_3112sell/model.joblib",
        # "sell_pca_path":  'AUDUSD_D1_3112sell/pca.joblib',
        "buy_features_path": "AUDUSD_D1_2023buy/feature_names.json",
        "sell_features_path": "AUDUSD_D1_3112sell/feature_names.json",
        "magic": 1004,
    }

    usdchf_config = {
        "symbol": "USDCHF",
        "login": 52118220,
        "password": "ZZ&$gcUEPtOg2j",
        "server": "ICMarketsEU-Demo",
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU9\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "1:2",
        "sell_risk_reward_ratio": "1:1",

        # Thresholds for buy/sell
        "buy_threshold": 0.55,
        "sell_threshold": 0.45,

        "use_dynamic_mean_candle_size": True,   # Set to False to use hardcoded mean_candle_size
        "mean_candle_size": 0.007,             # Used only if use_dynamic_mean_candle_size is False

        "buy_scaler_path": "USDCHF_D1_3112buy/scaler.joblib",
        "buy_model_path": "USDCHF_D1_3112buy/model.joblib",
        # "buy_pca_path":   'USDCHF_D1_3112buy/pca.joblib',
        "sell_scaler_path": "USDCHF_D1_3112sell/scaler.joblib",
        "sell_model_path": "USDCHF_D1_3112sell/model.joblib",
        # "sell_pca_path":  'USDCHF_D1_3112sell/pca.joblib',
        "buy_features_path": "USDCHF_D1_3112buy/feature_names.json",
        "sell_features_path": "USDCHF_D1_3112sell/feature_names.json",
        "magic": 1005,
    }

    configs = [
        eurusd_config, 
        gbpusd_config, 
        usdcad_config, 
        audusd_config, 
        #usdchf_config  # Uncomment to include USDCHF
    ]

    # Define the time range for historical data
    utc_from = datetime(2020, 1, 1, tzinfo=pytz.utc)
    utc_to = datetime.now(pytz.utc)

    # Initialize trade decisions dictionary (placeholder for future use)
    trade_decisions = {config['symbol']: {'buy': 0, 'sell': 0} for config in configs}
    threads = []

    # Start a separate thread for each currency pair
    for config in configs:
        thread = threading.Thread(target=process_pair, args=(config, utc_from, utc_to, trade_decisions))
        thread.start()
        threads.append(thread)

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()


In [1]:
import sys
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import pytz
import time
from joblib import load
import logging
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
import talib
import os
import threading
import subprocess
import psutil
import json

import warnings
warnings.filterwarnings("ignore", message="Dependency not available for matrix_profile")

# Initialize the lock globally
mt5_lock = threading.Lock()

logging.basicConfig(
    filename='trading_EURUSD_GBPUSD_USDCAD_AUDUSD_USDCHF_Buy_Sell_D1.log', 
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logging.basicConfig(level=logging.ERROR)

# Kill any existing MetaTrader 5 instances
def kill_mt5_instances():
    for process in psutil.process_iter():
        try:
            if "terminal64.exe" in process.name().lower():
                process.kill()
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass  # Ignore these exceptions

# Initialize connection to MetaTrader 5 with proper locking and multiple instances handling
def init_mt5_connection(login, password, server, terminal_path):
    mt5.shutdown()
    time.sleep(5)

    # Start MetaTrader 5 terminal for the specific account
    subprocess.Popen([terminal_path, '/portable', '/login', str(login), '/password', password, '/server', server])
    time.sleep(10)

    for _ in range(30):  # Poll for 30 seconds, check if MT5 is running
        if mt5.initialize(login=login, password=password, server=server):
            logging.info(f"Connected to MetaTrader 5 with login: {login}")
            print(f"Connected to MetaTrader 5 with login: {login}")
            return True
        time.sleep(1)

    logging.error(f"initialize() failed, error code = {mt5.last_error()}")
    sys.exit()

# Fetch historical data
def fetch_historical_data(symbol, timeframe, start_date, end_date):
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    
    # Check if data is empty or None
    if data is None or len(data) == 0:
        logging.error(f"No historical data found for {symbol} in the given date range.")
        print(f"No historical data found for {symbol}.")
        return None
    
    ohlc_data = pd.DataFrame(data)
    if 'time' not in ohlc_data.columns:
        logging.error(f"'time' column missing in the fetched data for {symbol}.")
        print(f"Error: 'time' column missing in the fetched data for {symbol}.")
        return None

    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    return ohlc_data[['time', 'open', 'high', 'low', 'close']]

# Calculate mean candle size dynamically
def calculate_mean_candle_size(df):
    df['candle_size'] = df['high'] - df['low']
    mean_candle_size = df['candle_size'].mean()
    return mean_candle_size

# Add rolling features
def add_rolling_features(df, window):
    df['rolling_mean_open'] = df['open'].rolling(window=window).mean()
    df['rolling_std_open'] = df['open'].rolling(window=window).std()
    df['rolling_mean_close'] = df['close'].rolling(window=window).mean()
    df['rolling_std_close'] = df['close'].rolling(window=window).std()
    df['rolling_mean_high'] = df['high'].rolling(window=window).mean()
    df['rolling_std_high'] = df['high'].rolling(window=window).std()
    df['rolling_mean_low'] = df['low'].rolling(window=window).mean()
    df['rolling_std_low'] = df['low'].rolling(window=window).std()
    return df

# Add lag features
def add_lag_features(df, lags):
    for lag in lags:
        df[f'open_lag_{lag}'] = df['open'].shift(lag)
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'high_lag_{lag}'] = df['high'].shift(lag)
        df[f'low_lag_{lag}'] = df['low'].shift(lag)
    return df

# Calculate indicators
def calculate_indicators(df):
    # EMAs
    df['EMA_9'] = talib.EMA(df['close'], timeperiod=9)
    df['EMA_21'] = talib.EMA(df['close'], timeperiod=21)
    df['EMA_50'] = talib.EMA(df['close'], timeperiod=50)
    
    # RSI
    df['RSI_9'] = talib.RSI(df['close'], timeperiod=9)
    df['RSI_14'] = talib.RSI(df['close'], timeperiod=14)
    df['RSI_21'] = talib.RSI(df['close'], timeperiod=21)
    
    # WILLR
    df['WILLR_15'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=15)
    df['WILLR_23'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=23)
    df['WILLR_42'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=42)
    df['WILLR_145'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=145)
    
    # SAR
    df['SAR'] = talib.SAR(df['high'], df['low'], acceleration=0.02, maximum=0.2)
    
    # Bollinger Bands
    df['BB_upper'], df['BB_middle'], df['BB_lower'] = talib.BBANDS(
        df['close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0
    )
    df['BB_width'] = df['BB_upper'] - df['BB_lower']
    
    # MACD
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(
        df['close'], fastperiod=12, slowperiod=26, signalperiod=9
    )
    
    # CCI
    df['CCI_14'] = talib.CCI(df['high'], df['low'], df['close'], timeperiod=14)
    
    return df

# Optional helper: example of how you might extract rolling features for a single column
def extract_rolling_features(df, column_name, symbol):
    df_melted = df[['time', column_name]].copy()
    df_melted["Symbols"] = symbol
    # Roll the data in a time-series manner
    df_rolled = roll_time_series(
        df_melted, 
        column_id="Symbols", 
        column_sort="time", 
        max_timeshift=20, 
        min_timeshift=5
    )
    # Extract features
    X = extract_features(
        df_rolled.drop("Symbols", axis=1),
        column_id="id",
        column_sort="time",
        column_value=column_name,
        impute_function=impute,
        show_warnings=False
    )
    # Map index back to original time
    X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
    return X

# Process data
def process_data_for_features(df, symbol):
    df = add_rolling_features(df, window=5)
    df = add_lag_features(df, lags=[1, 2, 3, 4, 5])
    df = df.dropna().reset_index(drop=True)

    # Extract rolling features
    X1  = extract_rolling_features(df, 'WILLR_15', symbol)
    X2  = extract_rolling_features(df, 'WILLR_42', symbol)
    X3  = extract_rolling_features(df, 'RSI_14',  symbol)
    X4  = extract_rolling_features(df, 'MACD_hist', symbol)
    X5  = extract_rolling_features(df, 'EMA_9', symbol)
    X6  = extract_rolling_features(df, 'EMA_21', symbol)
    X7  = extract_rolling_features(df, 'EMA_50', symbol)
    X9  = extract_rolling_features(df, 'RSI_9', symbol)
    X10 = extract_rolling_features(df, 'RSI_21', symbol)
    X11 = extract_rolling_features(df, 'WILLR_23', symbol)
    X12 = extract_rolling_features(df, 'WILLR_145', symbol)
    X13 = extract_rolling_features(df, 'SAR', symbol)
    X14 = extract_rolling_features(df, 'BB_width', symbol)
    X15 = extract_rolling_features(df, 'MACD_signal', symbol)
    X16 = extract_rolling_features(df, 'CCI_14', symbol)
    
    X = pd.concat(
        [X1, X2, X3, X4, X5, X6, X7, X9, X10, X11, X12, X13, X14, X15, X16],
        axis=1, 
        join='inner'
    ).dropna()

    df['time'] = pd.to_datetime(df['time'])
    df = df.set_index('time')

    # Align X with df’s index
    X = X[X.index.isin(df.index)]
    X = pd.concat([df, X], axis=1, join='inner')

    return X

def fetch_and_process_data(symbol, timeframe, signals, selected_features, start_date, end_date):
    logging.info(f"Fetching and processing data for {symbol}")

    df = fetch_historical_data(symbol, timeframe, start_date, end_date)
    if df is None or df.empty:
        logging.error(f"No historical data found for {symbol}.")
        return None, None

    mean_candle_size = calculate_mean_candle_size(df)
    df = calculate_indicators(df)
    combined_df = process_data_for_features(df, symbol)

    missing_features = [feat for feat in selected_features if feat not in combined_df.columns]
    for feat in missing_features:
        combined_df[feat] = 0

    combined_df = combined_df.dropna()
    return combined_df, mean_candle_size

def calculate_prices(entry_price, risk_reward_ratio, mean_candle_size, trade_type):
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part

    if trade_type == "Buy":
        sl_price = entry_price - risk_amount
        tp_price = entry_price + reward_amount
    elif trade_type == "Sell":
        sl_price = entry_price + risk_amount
        tp_price = entry_price - reward_amount
    else:
        raise ValueError(f"Invalid trade_type: {trade_type}. It must be 'Buy' or 'Sell'.")

    return sl_price, tp_price

def place_order(symbol, volume, sl_price, tp_price, trade_type, config, retry_attempts=3):
    for attempt in range(retry_attempts):
        with mt5_lock:
            tick_info = mt5.symbol_info_tick(symbol)
            if tick_info is None:
                logging.error(f"Failed to retrieve tick information for {symbol}.")
                return
            price = tick_info.ask if trade_type == "Buy" else tick_info.bid
            order_type = mt5.ORDER_TYPE_BUY if trade_type == "Buy" else mt5.ORDER_TYPE_SELL
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": volume,
                "type": order_type,
                "price": price,
                "sl": sl_price,
                "tp": tp_price,
                "deviation": 10,
                "magic": config["magic"],
                "comment": f"Python script {trade_type} order",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }

            result = mt5.order_send(request)
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                logging.info(f"{trade_type} order placed successfully for {symbol}.")
                print(f"{trade_type} order placed successfully for {symbol}.")
                return True
            else:
                logging.error(f"Order placement failed for {symbol}: {result.retcode}. Retrying...")
                print(f"Order placement failed for {symbol}: {result.retcode}. Retrying...")

        time.sleep(2)

    logging.error(f"Order failed after {retry_attempts} attempts for {symbol}")
    return False

def predict_and_gather_predictions(scaler,model,symbol,timeframe,start_date,end_date,features,trade_type,threshold=None,pca=None):
    """
    If pca is not None, we apply pca.transform() after scaler.transform().
    """
    signals = ['WILLR_15', 'WILLR_42']  # example signals or placeholders
    df_processed, mean_candle_size = fetch_and_process_data(symbol, timeframe, signals, features, start_date, end_date)
    if df_processed is None:
        logging.error(f"No valid data for {symbol}. Skipping predictions.")
        return None, mean_candle_size

    df_processed = df_processed.shift(periods=1, axis=0).dropna()
    df_processed = df_processed[features]

    scaled_data = scaler.transform(df_processed)

    if pca is not None:
        scaled_data = pca.transform(scaled_data)

    probas = model.predict_proba(scaled_data)[:, 1]
    custom_threshold = threshold if threshold is not None else 0.5
    y_pred = (probas >= custom_threshold).astype(int)
    pred = y_pred[-1] if len(y_pred) > 0 else 0

    X_df = pd.DataFrame(index=df_processed.index)
    X_df.index = pd.to_datetime(X_df.index)
    X_df.index = X_df.index.strftime('%Y-%m-%d %H:%M:%S')
    X_df.index.name = 'time'
    df_pred = pd.DataFrame(index=X_df.index)
    df_pred['prediction'] = y_pred
    df_pred.to_csv(f'pred_{symbol}_{trade_type}.csv')

    return pred, mean_candle_size

def decide_and_trade(buy_pred, sell_pred, symbol, volume, risk_reward_ratio, mean_candle_size, config):
    if buy_pred == 1 and sell_pred == 1:
        logging.info(f"Both Buy and Sell predictions are 1 for {symbol}. No trade will be taken.")
        return

    if buy_pred == 1 and sell_pred == 0:
        entry_price = mt5.symbol_info_tick(symbol).ask
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size, "Buy")
        place_order(symbol, volume, sl_price, tp_price, "Buy", config)

    if sell_pred == 1 and buy_pred == 0:
        entry_price = mt5.symbol_info_tick(symbol).bid
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size, "Sell")
        place_order(symbol, volume, sl_price, tp_price, "Sell", config)

def process_pair(config, utc_from, utc_to, trade_decisions,hr,min):
    logging.info(f"Processing pair: {config['symbol']} for Buy and Sell")

    buyscaler, buypca, buymodel = None, None, None
    sellscaler, sellpca, sellmodel = None, None, None

    try:
        buyscaler = load(config["buy_scaler_path"])
        buymodel  = load(config["buy_model_path"])
        with open(config["buy_features_path"], 'r') as f:
            buy_features = json.load(f)
        if "buy_pca_path" in config:
            buypca = load(config["buy_pca_path"])
    except Exception as e:
        logging.warning(f"Buy model/scaler/PCA missing for {config['symbol']}: {e}")
        print(f"Buy model/scaler/PCA missing for {config['symbol']}: {e}")
        buy_features = []

    try:
        if "sell_scaler_path" in config and "sell_model_path" in config:
            sellscaler = load(config["sell_scaler_path"])
            sellmodel  = load(config["sell_model_path"])
            with open(config["sell_features_path"], 'r') as f:
                sell_features = json.load(f)
            if "sell_pca_path" in config:
                sellpca = load(config["sell_pca_path"])
        else:
            logging.warning(f"Sell model or scaler missing for {config['symbol']}")
            print(f"Sell model or scaler missing for {config['symbol']}")
            sell_features = []
    except Exception as e:
        logging.warning(f"Error loading sell model for {config['symbol']}: {e}")
        print(f"Error loading sell model for {config['symbol']}: {e}")
        sell_features = []

    try:
        with mt5_lock:
            init_mt5_connection(config['login'], config['password'], config['server'], config['terminal_path'])

        while True:
            current_time = datetime.now(pytz.utc)

            if current_time.hour == hr and current_time.minute == min:  # example daily trading window
                buy_pred, mean_candle_size = None, None
                sell_pred = None

                # Only run if we have buy artifacts and features
                if buyscaler is not None and buymodel is not None and len(buy_features) > 0:
                    # Take buy_threshold from config
                    buy_threshold = config.get("buy_threshold", None)

                    buy_pred, mean_candle_size = predict_and_gather_predictions(
                        scaler=buyscaler,
                        model=buymodel,
                        symbol=config["symbol"],
                        timeframe=config["timeframe"],
                        start_date=utc_from,
                        end_date=utc_to,
                        features=buy_features,
                        trade_type="Buy",
                        threshold=buy_threshold,
                        pca=buypca
                    )

                # Only run if we have sell artifacts and features
                if sellscaler is not None and sellmodel is not None and len(sell_features) > 0:
                    # Take sell_threshold from config
                    sell_threshold = config.get("sell_threshold", None)

                    sell_pred, _ = predict_and_gather_predictions(
                        scaler=sellscaler,
                        model=sellmodel,
                        symbol=config["symbol"],
                        timeframe=config["timeframe"],
                        start_date=utc_from,
                        end_date=utc_to,
                        features=sell_features,
                        trade_type="Sell",
                        threshold=sell_threshold,
                        pca=sellpca
                    )

                if buy_pred is not None and sell_pred is not None:
                    decide_and_trade(
                        buy_pred=buy_pred,
                        sell_pred=sell_pred,
                        symbol=config["symbol"],
                        volume=config["volume"],
                        risk_reward_ratio=config["buy_risk_reward_ratio"],
                        mean_candle_size=mean_candle_size,
                        config=config
                    )

                logging.info(f"Finished trading for {config['symbol']} at {current_time} UTC")

            time.sleep(60)

    except KeyboardInterrupt:
        logging.info(f"Script terminated by user for {config['symbol']}.")
        print(f"Script terminated by user for {config['symbol']}.")
    finally:
        mt5.shutdown()
        logging.info(f"MetaTrader 5 connection closed for {config['symbol']}.")
        print(f"MetaTrader 5 connection closed for {config['symbol']}.")

def main():
    kill_mt5_instances()

    # Example configs with buy/sell thresholds specified
    eurusd_config = {
        "symbol": "EURUSD",
        "login": 52118058,
        "password": "tqLuM0wom!pL@P",
        "server": "ICMarketsEU-Demo",
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU5\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "1:2",
        "sell_risk_reward_ratio": "1:1",

        # Thresholds for buy/sell
        "buy_threshold": 0.6,
        "sell_threshold": 0.5,

        "use_dynamic_mean_candle_size": True,  # Set to False to use hardcoded mean_candle_size
        "mean_candle_size": 0.005,             # Used only if use_dynamic_mean_candle_size is False

        "buy_scaler_path": "EURUSD_D1_2023buy/scaler.joblib",
        "buy_model_path": "EURUSD_D1_2023buy/model.joblib",
        "buy_pca_path":   'EURUSD_D1_2023buy/pca.joblib',
        "sell_scaler_path": "EURUSD_D1_2023sell/scaler.joblib",
        "sell_model_path": "EURUSD_D1_2023sell/model.joblib",
        "sell_pca_path":  'EURUSD_D1_2023sell/pca.joblib',
        "buy_features_path": "EURUSD_D1_2023buy/feature_names.json",
        "sell_features_path": "EURUSD_D1_2023sell/feature_names.json",
        "magic": 1001,
    }

    gbpusd_config = {
        "symbol": "GBPUSD",
        "login": 52118161,
        "password": "t0cecVH!vqselU",
        "server": "ICMarketsEU-Demo",
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU6\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "2:2",
        "sell_risk_reward_ratio": "1:1",

        # Thresholds for buy/sell
        "buy_threshold": 0.6,
        "sell_threshold": 0.56,

        "use_dynamic_mean_candle_size": False,  # Set to True to calculate dynamically
        "mean_candle_size": 0.01,               # Used only if use_dynamic_mean_candle_size is False

        "buy_scaler_path": "GBPUSD_D1_2023buy/scaler.joblib",
        "buy_model_path": "GBPUSD_D1_2023buy/model.joblib",
        "buy_pca_path": "GBPUSD_D1_2023buy/pca.joblib",
        "sell_scaler_path": "GBPUSD_D1_2023sell/scaler.joblib",
        "sell_model_path": "GBPUSD_D1_2023sell/model.joblib",
        "sell_pca_path":  'GBPUSD_D1_2023sell/pca.joblib',
        "buy_features_path": "GBPUSD_D1_2023buy/feature_names.json",
        "sell_features_path": "GBPUSD_D1_2023sell/feature_names.json",
        "magic": 1002,
    }

    usdcad_config = {
        "symbol": "USDCAD",
        "login": 52118215,
        "password": "JU3fQ$XZ5FLeqQ",
        "server": "ICMarketsEU-Demo",
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU7\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "1:1",
        "sell_risk_reward_ratio": "2:3",

        # Thresholds for buy/sell
        "buy_threshold": 0.6,
        "sell_threshold": 0.6,

        "use_dynamic_mean_candle_size": False,   # Set to False to use hardcoded mean_candle_size
        "mean_candle_size": 0.0085,             # Used only if use_dynamic_mean_candle_size is False

        "buy_scaler_path": "USDCAD_D1_2023buy/scaler.joblib",
        "buy_model_path": "USDCAD_D1_2023buy/model.joblib",
        "buy_pca_path":   'USDCAD_D1_2023buy/pca.joblib',
        "sell_scaler_path": "USDCAD_D1_3112_Sell/scaler.joblib",
        "sell_model_path": "USDCAD_D1_3112_Sell/model.joblib",
        # "sell_pca_path":  'USDCAD_D1_3112_Sell/pca.joblib',
        "buy_features_path": "USDCAD_D1_2023buy/feature_names.json",
        "sell_features_path": "USDCAD_D1_3112_Sell/feature_names.json",
        "magic": 1003,
    }

    audusd_config = {
        "symbol": "AUDUSD",
        "login": 52118219,
        "password": "Taz$Ci50wB1SEx",
        "server": "ICMarketsEU-Demo",
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU8\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "1:1",
        "sell_risk_reward_ratio": "1:1",

        # Thresholds for buy/sell
        "buy_threshold": 0.6,
        "sell_threshold": 0.6,

        "use_dynamic_mean_candle_size": False,  # Set to True to use hardcoded mean_candle_size
        "mean_candle_size": 0.008,               # Used only if use_dynamic_mean_candle_size is False

        "buy_scaler_path": "AUDUSD_D1_2023buy/scaler.joblib",
        "buy_model_path": "AUDUSD_D1_2023buy/model.joblib",
        "buy_pca_path": "AUDUSD_D1_2023buy/pca.joblib",
        
        "sell_scaler_path": "AUDUSD_D1_2023sell/scaler.joblib",
        "sell_model_path": "AUDUSD_D1_2023sell/model.joblib",
        "sell_pca_path":  'AUDUSD_D1_2023sell/pca.joblib',
        "buy_features_path": "AUDUSD_D1_2023buy/feature_names.json",
        "sell_features_path": "AUDUSD_D1_2023sell/feature_names.json",
        "magic": 1004,
    }

    usdchf_config = {
        "symbol": "USDCHF",
        "login": 52118220,
        "password": "ZZ&$gcUEPtOg2j",
        "server": "ICMarketsEU-Demo",
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU9\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "1:2",
        "sell_risk_reward_ratio": "1:1",

        # Thresholds for buy/sell
        "buy_threshold": 0.55,
        "sell_threshold": 0.45,

        "use_dynamic_mean_candle_size": True,   # Set to False to use hardcoded mean_candle_size
        "mean_candle_size": 0.007,             # Used only if use_dynamic_mean_candle_size is False

        "buy_scaler_path": "USDCHF_D1_3112buy/scaler.joblib",
        "buy_model_path": "USDCHF_D1_3112buy/model.joblib",
        # "buy_pca_path":   'USDCHF_D1_3112buy/pca.joblib',
        "sell_scaler_path": "USDCHF_D1_3112sell/scaler.joblib",
        "sell_model_path": "USDCHF_D1_3112sell/model.joblib",
        # "sell_pca_path":  'USDCHF_D1_3112sell/pca.joblib',
        "buy_features_path": "USDCHF_D1_3112buy/feature_names.json",
        "sell_features_path": "USDCHF_D1_3112sell/feature_names.json",
        "magic": 1005,
    }

    configs = [
        eurusd_config, 
        gbpusd_config, 
        usdcad_config, 
        audusd_config, 
        #usdchf_config
    ]

    utc_from = datetime(2023, 1, 1, tzinfo=pytz.utc)
    utc_to = datetime.now()

    hr = 23
    min= 2

    trade_decisions = {config['symbol']: {'buy': 0, 'sell': 0} for config in configs}
    threads = []

    for config in configs:
        thread = threading.Thread(target=process_pair, args=(config, utc_from, utc_to, trade_decisions,hr,min))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main() 